This file shows how to implement tests for heteroscedasticity.

In [25]:
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.formula.api import glm
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams['figure.figsize'] = [10, 5]

In [26]:
import openpyxl
import seaborn as sns
import statsmodels.stats.diagnostic as dg
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.compat import lzip

In [24]:
df = pd.read_csv('/content/Wk6 Wage.csv')
df.head()
df.describe()

,wage,educ,exper,female,Metro,black,married,union,south,fulltime,l_wage
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000,1000.00000
mean,10.213020,13.285000,18.780000,0.494000,0.80800,0.088000,0.576000,0.165000,0.315000,0.87300,2.16701
std,6.246641,2.468171,11.318821,0.500214,0.39407,0.283437,0.494438,0.371366,0.464748,0.33314,0.55279
min,2.030000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.71000
25%,5.530000,12.000000,10.000000,0.000000,1.00000,0.000000,0.000000,0.000000,0.000000,1.00000,1.71000
50%,8.790000,13.000000,18.000000,0.000000,1.00000,0.000000,1.000000,0.000000,0.000000,1.00000,2.17500
75%,12.780000,16.000000,26.000000,1.000000,1.00000,0.000000,1.000000,0.000000,1.000000,1.00000,2.55000
max,60.190000,18.000000,52.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.00000,4.10000


Run the model with Lnwage against educ, exper, female and metro

In [20]:
mod=ols('l_wage~educ+exper+female+Metro',data=df).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                 l_wage   R-squared:                       0.337
Model:                            OLS   Adj. R-squared:                  0.335
Method:                 Least Squares   F-statistic:                     126.6
Date:                Mon, 12 May 2025   Prob (F-statistic):           2.21e-87
Time:                        05:40:25   Log-Likelihood:                -619.97
No. Observations:                1000   AIC:                             1250.
Df Residuals:                     995   BIC:                             1274.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4753      0.092      5.162      0.0

In [12]:
from statsmodels.stats.diagnostic import het_breuschpagan as breuschpagan
from statsmodels.stats.diagnostic import het_white as whites
from statsmodels.stats.diagnostic import linear_reset as reset

In [6]:
print('\nThe breuschpagan is the non-robust version and the Whites test includes all the interaction terms (not squares only)')

bp1=breuschpagan(mod.resid,mod.model.exog)
bp2=breuschpagan(mod.resid,mod.model.exog)
wh=whites(mod.resid,mod.model.exog)
print('\nbreuschpagan:','LM',bp1[0],'LM-pval',bp1[1],': F',bp1[2],'F-pval',bp1[3])
print('\nbreuschpagan:','LM',bp2[0],'LM-pval',bp2[1],': F',bp2[2],'F-pval',bp2[3])
print('\nwhites      :','LM',wh[0],'LM-pval',wh[1],': F',wh[2],'F-pval',wh[3])


The breuschpagan is the non-robust version and the Whites test includes all the interaction terms (not squares only)

breuschpagan: LM 20.51349920659662 LM-pval 0.0003953331662091952 : F 5.209600054219824 F-pval 0.00037253000550558854

breuschpagan: LM 20.51349920659662 LM-pval 0.0003953331662091952 : F 5.209600054219824 F-pval 0.00037253000550558854

whites      : LM 36.885592926692134 LM-pval 0.00023315046225020055 : F 3.1500307709440274 F-pval 0.00020108661490337684


Values differ a little compared to Gretl given how test statistic is calculated. But give the same result.

Below is an alternative way to run the RESET test

In [17]:
#You can select different Powers as in Gretl. This is as in the interactive session with power=2
print('\nreset',reset(mod,use_f=True,power=2))


reset <F test: F=1.4374136888222693, p=0.23084405728295307, df_denom=994, df_num=1>


Unable to reject H0 in this case.

In [19]:
#Alternative form of the Reset test can be obtained as
W=reset(mod)
print(W.summary())

<Wald test (chi2): statistic=9.681357070972993, p-value=0.00790169065465914, df_denom=2>


In [21]:
mod=ols('l_wage~educ+exper+female+Metro',data=df).fit()
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                 l_wage   R-squared:                       0.337
Model:                            OLS   Adj. R-squared:                  0.335
Method:                 Least Squares   F-statistic:                     126.6
Date:                Mon, 12 May 2025   Prob (F-statistic):           2.21e-87
Time:                        05:41:03   Log-Likelihood:                -619.97
No. Observations:                1000   AIC:                             1250.
Df Residuals:                     995   BIC:                             1274.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4753      0.092      5.162      0.0

Collecting the tests together

In [22]:
print('\nThe breuschpagan produced by Gretl is the "non-robust version" and the Whites test includes all the interaction terms')
bp1=breuschpagan(mod.resid,mod.model.exog,robust=False)
bp2=breuschpagan(mod.resid,mod.model.exog,robust=True)
wh=whites(mod.resid,mod.model.exog)
print('\nbreuschpagan:','LM',bp1[0],'LM-pval',bp1[1],': F',bp1[2],'F-pval',bp1[3])
print('\nbreuschpagan:','LM',bp2[0],'LM-pval',bp2[1],': F',bp2[2],'F-pval',bp2[3])
print('\nwhites      :','LM',wh[0],'LM-pval',wh[1],': F',wh[2],'F-pval',wh[3])
print('\nreset',reset(mod,use_f=True,power=2))


The breuschpagan produced by Gretl is the "non-robust version" and the Whites test includes all the interaction terms

breuschpagan: LM 22.989235769954803 LM-pval 0.00012725470832961516 : F 5.209600054219703 F-pval 0.00037253000550568715

breuschpagan: LM 20.51349920659662 LM-pval 0.0003953331662091952 : F 5.209600054219824 F-pval 0.00037253000550558854

whites      : LM 36.885592926692134 LM-pval 0.00023315046225020055 : F 3.1500307709440274 F-pval 0.00020108661490337684

reset <F test: F=1.4374136888222693, p=0.23084405728295307, df_denom=994, df_num=1>


Finally, how to implement robust standard errors

In [23]:
#There are several forms of robust standard errors, below is HC1, it is very slightly different to gretl
mod=ols('l_wage~educ+exper+female+Metro',data=df).fit(cov_type='HC1')
print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:                 l_wage   R-squared:                       0.337
Model:                            OLS   Adj. R-squared:                  0.335
Method:                 Least Squares   F-statistic:                     127.3
Date:                Mon, 12 May 2025   Prob (F-statistic):           9.11e-88
Time:                        05:44:22   Log-Likelihood:                -619.97
No. Observations:                1000   AIC:                             1250.
Df Residuals:                     995   BIC:                             1274.
Df Model:                           4                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4753      0.091      5.211      0.0